In this assignment, we follow the csv file that we created in Assignment 1 which includes games' names and metacritic score. We pick three games with highest metacritic score, and using twitter API to get twitter users' comments, and we build tables of tweets,tweets' hastags, twitter users, games, games' companies, game's producers, consumers. And we show the relationship by entities comsumerRgame, twitterRhastag and person.

# API part

In [291]:
#import pandas library
import pandas as pd

In [292]:
#Read the csv from last assignment
raw_csv = pd.read_csv('BasicInfo1.csv')

In [293]:
raw_csv.head()

Unnamed: 0                                        Name ReleaseDate  \
0         102                            Blueberry Garden      Jun-09   
1         104                             Darkest of Days      Sep-09   
2         108  Silent Hunter 5(r): Battle of the Atlantic      Mar-10   
3         109                                    Flotilla      Apr-10   
4         110                                Puzzle Agent      Jul-10   

   Metacritic  PriceInitial  
0          68          4.99  
1          51         19.99  
2          62          9.99  
3          72          9.99  
4          70          4.99

In [294]:
#Sort the game by metacritic score follow the descending order
raw_csv = raw_csv.sort_values('Metacritic',ascending = False)
raw_csv.head()

Unnamed: 0                Name  ReleaseDate  Metacritic  PriceInitial
1781        9792           Half-Life   Nov 8 1998          96          9.99
38           415  Grand Theft Auto V  Apr 13 2015          96         59.99
1663        9263         Half-Life 2  Nov 16 2004          96          9.99
299         1739        BioShock(tm)  Aug 21 2007          96         19.99
70           568            Portal 2  Apr 18 2011          95         19.99

In [295]:
#Choose the games with the top 3 highest metacritic score
top3score_csv = raw_csv.iloc[0:3]

In [296]:
top3score_csv

Unnamed: 0                Name  ReleaseDate  Metacritic  PriceInitial
1781        9792           Half-Life   Nov 8 1998          96          9.99
38           415  Grand Theft Auto V  Apr 13 2015          96         59.99
1663        9263         Half-Life 2  Nov 16 2004          96          9.99

In [297]:
#Only keep the name and metacritic score
keep_col = ['Name','Metacritic']
t3score_csv = top3score_csv[keep_col]
t3score_csv

Name  Metacritic
1781           Half-Life          96
38    Grand Theft Auto V          96
1663         Half-Life 2          96

In [298]:
t3score_csv.to_csv("top3scoregames.csv")

In [299]:
#import librarys and use the api to scrap information 
import tweepy
import json
import datetime
import warnings
warnings.filterwarnings('ignore')
consumerKey = '9oKAuFyGssLe8L9aCSTI0MRNU'
consumerSecret = 'UQJZvzAHF0no4K92nu4uoQWvt9c8PfCedNnBxMzbAlZnOe7FkF'
ACCESS_TOKEN = '520412304-rGe1islPm0TxbgO1YcJyw2T22V3lYDAxuNkFk5Md'
ACCESS_SECRET = '7XVyqlIVVB5SaHNNOSdEx6XPXP4MgGiB17P21ISkpXNtM'
auth = tweepy.OAuthHandler(consumer_key=consumerKey, 
    consumer_secret=consumerSecret)

#Connect to the Twitter API using the authentication
api = tweepy.API(auth)

In [300]:
# tweet to hashtag list, we will use it later
def tweettohashtaglist(tweet):
    hashtags = tweet._json.get("entities").get("hashtags")
    taglist = []
    for hashtag in hashtags:
        taglist.append(hashtag.get("text"))
    return taglist

In [301]:
# anyname to 30 days tweets list, we will use it later
def anynameto30daystweets(api, anyname):
    pgcount = 1
    allflag = False
    timenow = datetime.datetime.now()
    tweetlist = []
    while not allflag:
        tweets = api.user_timeline(anyname, page = pgcount, tweet_mode = 'extended')
        for tweet in tweets:
            if (timenow - tweet.created_at).days < 60:
                tweetlist.append(tweet)
            else:
                 allflag = True
        if not allflag:
            pgcount+=1
        else:
            return tweetlist

In [302]:
# hashtag to tweetlist, we will use it later
def hashtagtotweetlist(hashtag):
    tweets = []
    for tweet in tweepy.Cursor(api.search, q='#' + hashtag).items(2000):
        tweets.append(tweet)
    return tweets

# Setup the pandas table 

In [303]:
import pandas as pd
company = pd.DataFrame(columns=["companyId", "companyName", "twitterUserId"])
consumer = pd.DataFrame(columns=["personId"])
consumerRgame = pd.DataFrame(columns=["crgId", "personId", "gameId"])
game = pd.DataFrame(columns=["gameId", "gameName"])
hashtag = pd.DataFrame(columns=["hashtagId", "text"])
person = pd.DataFrame(columns=["personId", "personName", "twitterUserId"])
producer = pd.DataFrame(columns=["prdId", "personId", "gameId", "companyId"])
tweet = pd.DataFrame(columns=["tweetId", "twitterUserId", "content", "favoriteCount", "createdTime"])
tweetRhashtag = pd.DataFrame(columns=["trhId", "tweetId", "hashtagId"])
twitteruser = pd.DataFrame(columns=["twitterUserId", "name", "screenname", "location", "followersCount", "tweetsCount"])

In [304]:
# new twitter user appending function
# anyname to user
def anynametouser(anyname):
    user = api.get_user(anyname)
    return user
# user to user_id
def usertouserid(user):
    return user.id
# user to name
def usertoname(user):
    return user.name
# user to screen_name
def usertoscreenname(user):
    return user.screen_name
# user to location
def usertolocation(user):
    return user.location
# user to followers_count
def usertofollowerscount(user):
    return user.followers_count
# user to tweets_count
def usertotweetscount(user):
    return user.statuses_count


def appendtwitteruser(user):
    twitteruser.loc[twitteruser.shape[0]+1] = {"twitterUserId":usertouserid(user), 
                                               "name":usertoname(user), 
                                               "screenname":usertoscreenname(user), 
                                               "location":usertolocation(user), 
                                               "followersCount":usertofollowerscount(user), 
                                               "tweetsCount":usertotweetscount(user)}

Here we add basic attributes to companies, games, persons, producers.

In [305]:
company.loc[company.shape[0]+1] = {"companyId":1, "companyName":"Valve", "twitterUserId":usertouserid(anynametouser("Valve"))}
game.loc[game.shape[0]+1] = {"gameId":1, "gameName":"Half-Life"}
person.loc[person.shape[0]+1] = {"personId":1, "personName":"Gabe Newell", "twitterUserId":usertouserid(anynametouser("GabeLNewel"))}
producer.loc[producer.shape[0]+1] = {"prdId":1, "personId":1, "gameId":1, "companyId":1}
appendtwitteruser(anynametouser("GabeLNewel"))
appendtwitteruser(anynametouser("Valve"))
appendtwitteruser(anynametouser("rHalfLife"))

In [306]:
company.loc[company.shape[0]+1] = {"companyId":2, "companyName":"Rockstar", "twitterUserId":usertouserid(anynametouser("RockstarGames"))}
game.loc[game.shape[0]+1] = {"gameId":2, "gameName":"Grand Theft Auto V"}
person.loc[person.shape[0]+1] = {"personId":2, "personName":"Sam Houser", "twitterUserId":usertouserid(anynametouser("rock_dan_games"))}
producer.loc[producer.shape[0]+1] = {"prdId":2, "personId":2, "gameId":2, "companyId":2}
appendtwitteruser(anynametouser("rock_dan_games"))
appendtwitteruser(anynametouser("RockstarGames"))
appendtwitteruser(anynametouser("RockstarGTAV"))
game.loc[game.shape[0]+1] = {"gameId":3, "gameName":"Half-Life 2"}
producer.loc[producer.shape[0]+1] = {"prdId":3, "personId":1, "gameId":3, "companyId":1}

In [307]:
# new tweet appending function
# tweet to tweet_id
def tweettotweetid(tweet):
    return tweet._json.get("id")
# tweet to user_id
def tweettouserid(tweet):
    return tweet._json.get("user").get("id")
# tweet to text
def tweettotext(tweet):
    return tweet._json.get("full_text")
# tweet to favorite_count
def tweettofavoritecount(tweet):
    return tweet._json.get("favorite_count")
# tweet to created_time
def tweettocreatedtime(tweet):
    return tweet.created_at



def appendtweet(twt):
    if tweet[tweet["tweetId"].isin([tweettotweetid(twt)])].shape[0] == 0:
        tweet.loc[tweet.shape[0]+1] = {"tweetId":tweettotweetid(twt), 
                                       "twitterUserId":tweettouserid(twt), 
                                       "content":tweettotext(twt), 
                                       "favoriteCount":tweettofavoritecount(twt), 
                                       "createdTime": tweettocreatedtime(twt)}
        for tag in tweettohashtaglist(twt):
            if hashtag[hashtag["text"].isin([tag])].shape[0] == 0:
                hashtag.loc[hashtag.shape[0]+1] = {"hashtagId":hashtag.shape[0]+1, "text":tag}
            tweetRhashtag.loc[tweetRhashtag.shape[0]+1] = {"trhId":tweetRhashtag.shape[0]+1, 
                                                           "tweetId":tweettotweetid(twt), 
                                                           "hashtagId":hashtag[hashtag["text"].isin([tag])].loc[:, ["hashtagId"]].values[0][0]}

In [308]:
# save producers' or companys' tweets in 30 days
def saveproducercompanytweet(anyname):
    for twt0 in anynameto30daystweets(api, anyname):
            appendtweet(twt0)

In [309]:
try:
    saveproducercompanytweet("GabeLNewel")
    saveproducercompanytweet("Valve")
    saveproducercompanytweet("rHalfLife")
    saveproducercompanytweet("rock_dan_games")
    saveproducercompanytweet("RockstarGames")
    saveproducercompanytweet("RockstarGTAV")
except tweepy.error.TweepError:
    pass

In [310]:
# save tweets based on the hashtag of gameName
def savehashtagtweet(gameName0):
    for twt0 in hashtagtotweetlist(gameName0):
        appendtweet(twt0)
        if twitteruser[twitteruser["twitterUserId"].isin([tweettouserid(twt0)])].shape[0] == 0:
            twitteruser.loc[twitteruser.shape[0]+1] = {"twitterUserId":tweettouserid(twt0), 
                                                       "name":usertoname(anynametouser(tweettouserid(twt0))), 
                                                       "screenname":usertoscreenname(anynametouser(tweettouserid(twt0))), 
                                                       "location":usertolocation(anynametouser(tweettouserid(twt0))), 
                                                       "followersCount":usertofollowerscount(anynametouser(tweettouserid(twt0))), 
                                                       "tweetsCount":usertotweetscount(anynametouser(tweettouserid(twt0)))}
            person.loc[person.shape[0]+1] = {"personId":person.shape[0]+1, 
                                             "personName":usertoscreenname(anynametouser(tweettouserid(twt0))), 
                                             "twitterUserId":tweettouserid(twt0)}
            consumer.loc[consumer.shape[0]+1] = {"personId":person.shape[0]}
        consumerRgame.loc[consumerRgame.shape[0]+1] = {
            "crgId":consumerRgame.shape[0]+1,
            "personId":person[person["twitterUserId"].isin([tweettouserid(twt0)])].loc[:, ["personId"]].values[0][0],
            "gameId":game[game["gameName"].isin([gameName0])].loc[:, ["gameId"]].values[0][0]
            }
        consumerRgame.drop_duplicates(subset=["personId","gameId"], keep='first', inplace=False)

In [311]:
try:
    savehashtagtweet("Half-Life")
    savehashtagtweet("Grand Theft Auto V")
    savehashtagtweet("Half-Life 2")
except tweepy.error.TweepError:
    pass

# Here we get csv file

In [312]:
# company = pd.DataFrame(columns=["companyId", "companyName", "twitterUserId"])
company

companyId companyName twitterUserId
1         1       Valve       9317742
2         2    Rockstar      29758446

All information is showed, no 0, no null value, so it is already audited

In [313]:
#Make company a csv
company.to_csv(r'company.csv',index=False, encoding="utf-8-sig")

In [314]:
# consumer = pd.DataFrame(columns=["personId"])
consumer

personId
1          3
2          4
3          5
4          6
5          7
6          8
7          9
8         10
9         11
10        12
11        13
12        14
13        15
14        16
15        17
16        18
17        19
18        20
19        21
20        22
21        23
22        24
23        25
24        26
25        27
26        28
27        29
28        30
29        31
30        32
..       ...
119      121
120      122
121      123
122      124
123      125
124      126
125      127
126      128
127      129
128      130
129      131
130      132
131      133
132      134
133      135
134      136
135      137
136      138
137      139
138      140
139      141
140      142
141      143
142      144
143      145
144      146
145      147
146      148
147      149
148      150

[148 rows x 1 columns]

In [315]:
consumer.shape

(148, 1)

In [316]:
consumer.isnull().values.any()

False

No null value, and no 0, we make costumer a csv

In [317]:
consumer.to_csv(r'costumer.csv',index=False, encoding="utf-8-sig")

In [318]:
# consumerRgame = pd.DataFrame(columns=["crgId", "personId", "gameId"])
consumerRgame

crgId personId gameId
1       1        3      1
2       2        4      1
3       3        5      1
4       4        6      1
5       5        7      1
6       6        8      1
7       7        9      1
8       8       10      1
9       9       11      1
10     10       12      1
11     11       13      1
12     12       14      1
13     13       15      1
14     14       16      1
15     15       17      1
16     16       18      1
17     17       19      1
18     18       20      1
19     19       21      1
20     20       22      1
21     21       23      1
22     22       24      1
23     23       25      1
24     24       26      1
25     25       27      1
26     26       18      1
27     27       28      1
28     28       29      1
29     29       28      1
30     30       30      1
..    ...      ...    ...
137   137      125      1
138   138      112      1
139   139      126      1
140   140      111      1
141   141      127      1
142   142      111      1
143   143      128      1
144   144      129      1
145   145      130      1
146   146      131      1
147   147      132      1
148   148      133      1
149   149      134      1
150   150      135      1
151   151      136      1
152   152      137      1
153   153      138      1
154   154      139      1
155   155       39      1
156   156      140      1
157   157      141      1
158   158      142      1
159   159      143      1
160   160      144      1
161   161      145      1
162   162      146      1
163   163      147      1
164   164      148      1
165   165      149      1
166   166      150      1

[166 rows x 3 columns]

In [319]:
consumerRgame.shape

(166, 3)

In [320]:
consumerRgame.isnull().values.any()

False

No null values, we make consumerRgame a csv

In [321]:
consumerRgame.to_csv(r'consumerRgame.csv',index=False, encoding="utf-8-sig")

In [322]:
# game = pd.DataFrame(columns=["gameId", "gameName")
game

gameId            gameName
1      1           Half-Life
2      2  Grand Theft Auto V
3      3         Half-Life 2

All information is showed, no null values, no 0, so it is already audited

In [323]:
#make game a csv
game.to_csv(r'game.csv',index=False, encoding="utf-8-sig")

In [324]:
# hashtag = pd.DataFrame(columns=["hashtagId", "text"])
hashtag

hashtagId         text
1          1       eSport
2          2        Dota2
3          3         CSGO
4          4         PUBG
5          5  Hearthstone
6          6     Fortnite
7          7   AudibleApp
8          8      FoxNews
9          9           VR
10        10           XR
11        11         UXXR
12        12     snotquiz

All information is showed, no null values, no 0, so it is already audited

In [325]:
hashtag.to_csv(r'hashtag.csv',index=False, encoding="utf-8-sig")

In [326]:
# person = pd.DataFrame(columns=["personId", "personName", "twitterUserId"])
person

personId       personName        twitterUserId
1          1      Gabe Newell   979715337892630528
2          2       Sam Houser            588658717
3          3         idbepvya  1092534179739377666
4          4       hecksmoney  1230988107450851329
5          5       CHChistris            293140450
6          6  Sansfunnybonesr           4550941577
7          7      Waddedmoose            720775608
8          8        Luftspace           1678834922
9          9     Ryannessence  1041537869867888640
10        10        combotent             23677622
11        11         radioday             23790825
12        12        Com_Raven            108961481
13        13        Liammch12           4883133520
14        14    CrispineAgata   923648834269581312
15        15    griffingold55   813244354059993089
16        16          Fringon            117573476
17        17         NotTayyy           2794418748
18        18     TN_Collision            369762356
19        19  liltravindacity  1132034177455534081
20        20        Baka_idk1  1096440578458439687
21        21           Naimvb            196731509
22        22    techgoodjames   708148803979886592
23        23           hup333            138845190
24        24   CathedralWater           2781524100
25        25     BradentendoN  1087228878836264965
26        26     rashernvideo   731003090720067584
27        27   GingerWellyton   776278190041948160
28        28      AtomElwood2  1149945059527102465
29        29            Asp3F   941241578584203264
30        30  ValveNewsNetwor           2859077596
..       ...              ...                  ...
121      121      timdillon78             19823203
122      122           psb_dc           2908107542
123      123         microoci            234851506
124      124  YeahCallTheCops  1199779480975396866
125      125  WavingFromShore  1217050527323824129
126      126     kweenuhsheba  1185965196289949696
127      127    JosemaComites           2822809056
128      128    rllyxwhoxamxi            444548943
129      129        monroyspc            170883245
130      130     meninoUehara  1059898305222795264
131      131        Pixel_Cod   764850106524893185
132      132        Spirit__Z            390347118
133      133   Levi_the_Doggo           2433954415
134      134        ruudy_208   708713067660578816
135      135          junuve1  1029534399090618368
136      136  SeadogDriftwood             30165386
137      137     EllianRubina            108764433
138      138     KittyCaustic  1015112450222579713
139      139     beflosquotes  1124864315859918849
140      140         hirhir13           2394525764
141      141    NotMe87577664           1357328737
142      142   Kimmie_classic            301789842
143      143   Sparky91783238  1077977684125368320
144      144         ToYo2495            470525667
145      145   TheLizardizzle           1667581609
146      146           Sovi8_            951954090
147      147          _thoeni           1060708562
148      148     DeeristDandy  1128855616926224385
149      149    BlackMesaDevs             56029476
150      150    GummySkeleton             16509293

[150 rows x 3 columns]

In [327]:
person.shape

(150, 3)

In [328]:
person.isnull().values.any()

False

No null values, we make it a csv

In [329]:
person.to_csv(r'person.csv',index=False, encoding="utf-8-sig")

In [330]:
# producer = pd.DataFrame(columns=["prdId", "personId", "gameId", "companyId"])
producer

prdId personId gameId companyId
1     1        1      1         1
2     2        2      2         2
3     3        1      3         1

All information showed, no null values or 0, we make it a csv

In [331]:
producer.to_csv(r'producer.csv',index=False, encoding="utf-8-sig")

In [332]:
# tweet = pd.DataFrame(columns=["tweetId", "twitterUserId", "content", "favoriteCount", "createdTime"])
tweet

tweetId        twitterUserId content favoriteCount  \
1    1231379753132797953  1092534179739377666    None             0   
2    1231379605946294273  1230988107450851329    None             0   
3    1231379399586521090            293140450    None             0   
4    1231379364849188864           4550941577    None             0   
5    1231378699112648706            720775608    None             1   
6    1231378665491124224           1678834922    None             0   
7    1231376918164967424  1041537869867888640    None             0   
8    1231376900330622976             23677622    None             0   
9    1231376878251991040             23790825    None             0   
10   1231376823155642369            108961481    None             0   
11   1231376597766332417           4883133520    None             0   
12   1231376380769841153   923648834269581312    None             0   
13   1231375704325054464   813244354059993089    None             0   
14   1231374933890125826            117573476    None             0   
15   1231374285886959616           2794418748    None             0   
16   1231374163077685249            369762356    None             0   
17   1231374152734593024  1132034177455534081    None             0   
18   1231374100741984262  1096440578458439687    None             1   
19   1231374073004986368            196731509    None             0   
20   1231373958441730048   708148803979886592    None             0   
21   1231373908772872193            138845190    None             0   
22   1231373391791259648           2781524100    None             0   
23   1231373183913099264  1087228878836264965    None             1   
24   1231372982729109504   731003090720067584    None             0   
25   1231372969181474816   776278190041948160    None             0   
26   1231372872465162243            369762356    None             0   
27   1231372413255872513  1149945059527102465    None             0   
28   1231372306599022593   941241578584203264    None             0   
29   1231372171001286656  1149945059527102465    None             0   
30   1231372013610115077           2859077596    None            44   
..                   ...                  ...     ...           ...   
138  1231341110364188673  1037816383827050496    None             0   
139  1231340884983193600  1185965196289949696    None             0   
140  1231340828930514944   983504241582051328    None             0   
141  1231340560432205824           2822809056    None             0   
142  1231340544581939201   983504241582051328    None             0   
143  1231340443595628548            444548943    None             0   
144  1231340304302845958            170883245    None             0   
145  1231340293817061377  1059898305222795264    None             1   
146  1231339911556538369   764850106524893185    None             0   
147  1231339790953590785            390347118    None             0   
148  1231339746863013888           2433954415    None             0   
149  1231339699194728448   708713067660578816    None             0   
150  1231339578734325763  1029534399090618368    None             0   
151  1231339429303812101             30165386    None             0   
152  1231339428511207425            108764433    None             1   
153  1231339395082604550  1015112450222579713    None             4   
154  1231339326098694144  1124864315859918849    None             0   
155  1231339309531312128   908077539578531841    None             0   
156  1231339307413032960           2394525764    None             0   
157  1231339162005114887           1357328737    None             0   
158  1231339147392143360            301789842    None             0   
159  1231339075111735296  1077977684125368320    None             0   
160  1231339055092289536            470525667    None             0   
161  1231338740540293120           1667581609    None             1   
162  1231338698580602884            95195

In [333]:
tweet.shape

(167, 5)

In [334]:
tweet.isnull().values.any()

True

In [335]:
tweet.isnull().sum()

tweetId            0
twitterUserId      0
content          167
favoriteCount      0
createdTime        0
dtype: int64

From previous step we see all the content is null, so we no longer keep the attributes "content" in tweet table

In [336]:
keep_col = ['tweetId','twitterUserId','favoriteCount','createdTime']
tweet = tweet[keep_col]

In [337]:
tweet

tweetId        twitterUserId favoriteCount  \
1    1231379753132797953  1092534179739377666             0   
2    1231379605946294273  1230988107450851329             0   
3    1231379399586521090            293140450             0   
4    1231379364849188864           4550941577             0   
5    1231378699112648706            720775608             1   
6    1231378665491124224           1678834922             0   
7    1231376918164967424  1041537869867888640             0   
8    1231376900330622976             23677622             0   
9    1231376878251991040             23790825             0   
10   1231376823155642369            108961481             0   
11   1231376597766332417           4883133520             0   
12   1231376380769841153   923648834269581312             0   
13   1231375704325054464   813244354059993089             0   
14   1231374933890125826            117573476             0   
15   1231374285886959616           2794418748             0   
16   1231374163077685249            369762356             0   
17   1231374152734593024  1132034177455534081             0   
18   1231374100741984262  1096440578458439687             1   
19   1231374073004986368            196731509             0   
20   1231373958441730048   708148803979886592             0   
21   1231373908772872193            138845190             0   
22   1231373391791259648           2781524100             0   
23   1231373183913099264  1087228878836264965             1   
24   1231372982729109504   731003090720067584             0   
25   1231372969181474816   776278190041948160             0   
26   1231372872465162243            369762356             0   
27   1231372413255872513  1149945059527102465             0   
28   1231372306599022593   941241578584203264             0   
29   1231372171001286656  1149945059527102465             0   
30   1231372013610115077           2859077596            44   
..                   ...                  ...           ...   
138  1231341110364188673  1037816383827050496             0   
139  1231340884983193600  1185965196289949696             0   
140  1231340828930514944   983504241582051328             0   
141  1231340560432205824           2822809056             0   
142  1231340544581939201   983504241582051328             0   
143  1231340443595628548            444548943             0   
144  1231340304302845958            170883245             0   
145  1231340293817061377  1059898305222795264             1   
146  1231339911556538369   764850106524893185             0   
147  1231339790953590785            390347118             0   
148  1231339746863013888           2433954415             0   
149  1231339699194728448   708713067660578816             0   
150  1231339578734325763  1029534399090618368             0   
151  1231339429303812101             30165386             0   
152  1231339428511207425            108764433             1   
153  1231339395082604550  1015112450222579713             4   
154  1231339326098694144  1124864315859918849             0   
155  1231339309531312128   908077539578531841             0   
156  1231339307413032960           2394525764             0   
157  1231339162005114887           1357328737             0   
158  1231339147392143360            301789842             0   
159  1231339075111735296  1077977684125368320             0   
160  1231339055092289536            470525667             0   
161  1231338740540293120           1667581609             1   
162  1231338698580602884            951954090             0   
163  1231338349455200259           1060708562             0   
164  1231338278042951681  1128855616926224385             0   
165  1231338206383104000             56029476             0   
166  1231338076653273089             16509293             3   
167  1231337926317035525             16736535            54   

            createdTime  
1   2020-02-23 00:46:25  
2   2020-02-23 00:45:50  
3   2020-02-23 00:45:01  
4   20

In [338]:
#make tweet a csv
tweet.to_csv(r'tweet.csv',index=False, encoding="utf-8-sig")

In [339]:
# tweetRhashtag = pd.DataFrame(columns=["trhId", "tweetId", "hashtagId"])
tweetRhashtag

trhId              tweetId hashtagId
1      1  1231370680085598209         1
2      2  1231370680085598209         2
3      3  1231370680085598209         3
4      4  1231370680085598209         4
5      5  1231370680085598209         5
6      6  1231370680085598209         6
7      7  1231370253130551296         7
8      8  1231365342003449869         8
9      9  1231362101899296769         9
10    10  1231362101899296769        10
11    11  1231362101899296769        11
12    12  1231360609003999232         8
13    13  1231357468061716481         8
14    14  1231352984287748096         1
15    15  1231352984287748096         2
16    16  1231352984287748096         3
17    17  1231352984287748096         4
18    18  1231352984287748096         5
19    19  1231352581424926721        12

All information is showed, no null or 0, so it is already audited

In [340]:
#make tweetRhashtag a csv
tweetRhashtag.to_csv(r'tweetRhashtag.csv',index=False, encoding="utf-8-sig")

In [341]:
# twitteruser = pd.DataFrame(columns=["twitterUserId", "name", "screenname", "location", "followersCount", "tweetsCount"])
twitteruser

twitterUserId                                               name  \
1     979715337892630528                               Gabe L. Newell Valve   
2                9317742                                          Not Valve   
3    1197309773290586112                                         r/HalfLife   
4              588658717                                        dan houser    
5               29758446                                     Rockstar Games   
6              546825102                                 Grand Theft Auto V   
7    1092534179739377666                                           idbepvya   
8    1230988107450851329                                                 HM   
9              293140450                                  Chris (E)【=◈︿◈=】🌐   
10            4550941577                                     Sansfunnybones   
11             720775608                                         Lil' Meaty   
12            1678834922                                               Luft   
13   1041537869867888640                                             Ryanna   
14              23677622                                          myles.app   
15              23790825                                            The NSU   
16             108961481                                       Marcel Hatam   
17            4883133520                               Liam McFPS on PC 🇮🇪🔥   
18    923648834269581312                                     Crispine Agata   
19    813244354059993089                                 The Loss Soldier's   
20             117573476                                    Pedro Alcântara   
21            2794418748                                               Tayy   
22             369762356                                          Collision   
23   1132034177455534081                                           Web.Ster   
24   1096440578458439687                                   (not) baka idk 💛   
25             196731509                                          Naim V.B.   
26    708148803979886592                               James (HLA MARCH 23)   
27             138845190                                                hup   
28            2781524100                                               Blue   
29   1087228878836264965                                 Bradentendo vol. 2   
30    731003090720067584                                            Rashern   
..                   ...                                                ...   
125             19823203                                         Tim Dillon   
126           2908107542                                         Theo - 劉䂀曼   
127            234851506                                       Microoci.com   
128  1199779480975396866     🔞Power bottom Bruno rights🔞 (COMMISSIONS OPEN)   
129  1217050527323824129                                           Dan Hale   
130  1185965196289949696                                                rah   
131           2822809056                                             J-sema   
132            444548943                                              Marín   
133            170883245                                        Nery Monroy   
134  1059898305222795264                                             Neruda   
135   764850106524893185                                           PixelCod   
136            390347118                                   Alex 'Spirit' J.   
137           2433954415                      WATCH THE GODDAMN SONIC MOVIE   
138   708713067660578816                                               Rudy   
139  1029534399090618368                                             junuve   
140             30165386  I am no prophet; I am no warrior - I am a musi...   
141            108764433                                      Ellian Rubina   
142  1015112450222579713                                    MolitovMichelle   
143  1124864315859918849                                real 

In [342]:
twitteruser.shape

(154, 6)

In [343]:
twitteruser.isnull().values.any()

False

No null values, and some of tweetscount is 0. But 0 for tweetcount is a meaningful information, since it shows that this user has not tweeted. So we keep the 0

In [344]:
twitteruser.to_csv(r'tweetuser.csv',index=False, encoding="utf-8-sig")

# Here we generate our database

In [385]:
#import csv and sqlite3 library
import csv
import sqlite3

Here we read the data from the csv we have created

In [346]:
company = pd.read_csv(r'company.csv', encoding="utf-8-sig")
consumer = pd.read_csv('costumer.csv', encoding="utf-8-sig")
consumerRgame = pd.read_csv('consumerRgame.csv', encoding="utf-8-sig")
game = pd.read_csv('game.csv', encoding="utf-8-sig")
hashtag = pd.read_csv('hashtag.csv', encoding="utf-8-sig")
person = pd.read_csv('person.csv', encoding="utf-8-sig")
producer = pd.read_csv('producer.csv', encoding="utf-8-sig")
tweet = pd.read_csv('tweet.csv', encoding="utf-8-sig")
tweetRhashtag = pd.read_csv('tweetRhashtag.csv', encoding="utf-8-sig")
twitteruser = pd.read_csv('tweetuser.csv', encoding="utf-8-sig")



Make data lists

In [347]:
companyList = company.values.tolist()
consumerList = consumer.values.tolist()
consumerRgameList = consumerRgame.values.tolist()
gameList = game.values.tolist()
hashtagList = hashtag.values.tolist()
personList = person.values.tolist()
producerList = producer.values.tolist()
tweetList = tweet.values.tolist()
tweetRhashtagList = tweetRhashtag.values.tolist()
twitteruserList = twitteruser.values.tolist()

In [348]:
con = sqlite3.connect('assignment2.db')

In [349]:
c = con.cursor()

We create tables here into our database

In [350]:
c.execute("""CREATE TABLE company(
                   companyId INTEGER(20) PRIMARY KEY,
                   companyName VARCHAR(255),
                   twitterUserId INTEGER(255),
                   CONSTRAINT name_uni UNIQUE (companyId, twitterUserId)
                   
                   )""")

In [351]:
c.execute("""CREATE TABLE consumer(
                   personId INTEGER(20) PRIMARY KEY,
                   CONSTRAINT name_uni UNIQUE (personId)
                   )""")

In [352]:
c.execute("""CREATE TABLE consumerRgame(
                   crgId INTEGER(20) PRIMARY KEY,
                   personId INTEGER(20),
                   gameId INTEGER(20),
                   CONSTRAINT name_uni UNIQUE (crgId)
                   )""")

In [353]:
c.execute("""CREATE TABLE game(
                   gameId INTEGER(20) PRIMARY KEY,
                   gameName VARCHAR(255),
                   CONSTRAINT name_uni UNIQUE (gameId)
                   )""")

In [354]:
c.execute("""CREATE TABLE hashtag(
                   hashtagId INTEGER(20) PRIMARY KEY,
                   text VARCHAR(255),
                   CONSTRAINT name_uni UNIQUE (hashtagId, text)
                   )""")

In [355]:
c.execute("""CREATE TABLE person(
                   personId INTEGER(20) PRIMARY KEY,
                   personName VARCHAR(255),
                   twitterUserId INTEGER(255),
                   CONSTRAINT name_uni UNIQUE (personId, twitterUserId)
                   )""")

In [356]:
c.execute("""CREATE TABLE producer(
                   prdId INTEGER(20) PRIMARY KEY,
                   personId INTEGER(20),
                   gameId INTEGER(20),
                   companyId INTEGER(20),
                   CONSTRAINT name_uni UNIQUE (prdId)
                   )""")

In [357]:
c.execute("""CREATE TABLE tweet(
                   tweetId INTEGER(255) PRIMARY KEY,
                   twitterUserId INTEGER(255),
                   favoriteCount INTEGER(20),
                   createdTime DATE(20),
                   CONSTRAINT name_uni UNIQUE (tweetId)
                   )""")

In [358]:
c.execute("""CREATE TABLE tweetRhashtag(
                   trhId INTEGER(20) PRIMARY KEY,
                   tweetId INTEGER(255),
                   hashtagId INTEGER(20),
                   CONSTRAINT name_uni UNIQUE (trhId)
                   )""")

In [359]:
c.execute("""CREATE TABLE twitteruser(
                   twitterUserId INTEGER(255) PRIMARY KEY,
                   name VARCHAR(255),
                   screenname VARCHAR(255),
                   location VARCHAR(255),
                   followersCount INTEGER(20),
                   tweetsCount INTEGER(20),
                   CONSTRAINT name_uni UNIQUE (twitterUserId)
                   )""")

# Insert values from our lists into tables

Here we insert values into our tables into databases

In [360]:
c.executemany("""INSERT INTO company VALUES(?,?,?)""", companyList)
c.executemany("""INSERT INTO consumer VALUES(?)""", consumerList)
c.executemany("""INSERT INTO consumerRgame VALUES(?,?,?)""", consumerRgameList)
c.executemany("""INSERT INTO game VALUES(?,?)""", gameList)
c.executemany("""INSERT INTO hashtag VALUES(?,?)""", hashtagList)
c.executemany("""INSERT INTO person VALUES(?,?,?)""", personList)
c.executemany("""INSERT INTO producer VALUES(?,?,?,?)""", producerList)
c.executemany("""INSERT INTO tweet VALUES(?,?,?,?)""", tweetList)
c.executemany("""INSERT INTO tweetRhashtag VALUES(?,?,?)""", tweetRhashtagList)
c.executemany("""INSERT INTO twitteruser VALUES(?,?,?,?,?,?)""", twitteruserList)

# Test Case

Select all companies

In [361]:
c.execute("""SELECT *
             FROM company
          """)
for row in c.fetchall():
    print(row)

(1, 'Valve', 9317742)
(2, 'Rockstar', 29758446)


Select all producers

In [363]:
c.execute("""SELECT *
             FROM producer
          """)
for row in c.fetchall():
    print(row)

(1, 1, 1, 1)
(2, 2, 2, 2)
(3, 1, 3, 1)


Select all the screennames from twitterusers

In [364]:
c.execute("""SELECT screenname
             FROM twitteruser
          """)
for row in c.fetchall():
    print(row)

('GabeLNewel',)
('valve',)
('rHalfLife',)
('rock_dan_games',)
('RockstarGames',)
('RockstarGTAV',)
('idbepvya',)
('hecksmoney',)
('CHChistris',)
('Sansfunnybonesr',)
('Waddedmoose',)
('Luftspace',)
('Ryannessence',)
('combotent',)
('radioday',)
('Com_Raven',)
('Liammch12',)
('CrispineAgata',)
('griffingold55',)
('Fringon',)
('NotTayyy',)
('TN_Collision',)
('liltravindacity',)
('Baka_idk1',)
('Naimvb',)
('techgoodjames',)
('hup333',)
('CathedralWater',)
('BradentendoN',)
('rashernvideo',)
('GingerWellyton',)
('AtomElwood2',)
('Asp3F',)
('ValveNewsNetwor',)
('theaahava',)
('Jimmy_the_Newb',)
('CRAB_ROBOT',)
('DendrocacaliaJp',)
('GregFromFG',)
('BrennenGlover',)
('Angelakham',)
('maddavid890',)
('Luna_Best_Pone',)
('ValveArchive',)
('MitchyD',)
('DiBukowski',)
('tuckernuckjim',)
('ShadytoF',)
('Studpardee',)
('VRGirlWaluigi',)
('nuttyhill',)
('Dangermouseme',)
('DeansDailyDoses',)
('OregonGamer541',)
('PyroFreeman',)
('florin15791422',)
('GOOGLE_FLEX',)
('Marlamin',)
('uxxrdesign',)
('Pe

# User case

Get twitteruser' userid order by their screenname in descending order

In [366]:

c.execute("""SELECT twitterUserId
             FROM twitteruser
             ORDER BY screenname DESC
          """)
for row in c.fetchall():
    print(row)

(324179558,)
(2159060443,)
(2911221,)
(9317742,)
(993764623487651840,)
(2365155024,)
(15100976,)
(14785318,)
(19823203,)
(761770608191569928,)
(979972483,)
(708148803979886592,)
(64878397,)
(1125077400604348417,)
(708713067660578816,)
(588658717,)
(39543561,)
(444548943,)
(3750034637,)
(731003090720067584,)
(23790825,)
(1197309773290586112,)
(2908107542,)
(93062399,)
(1105216884331479040,)
(34591006,)
(170883245,)
(234851506,)
(1059898305222795264,)
(3366041229,)
(189297387,)
(1006342334936739840,)
(129360365,)
(863575244769865728,)
(1132034177455534081,)
(1185965196289949696,)
(2376856940,)
(1029534399090618368,)
(1092534179739377666,)
(138845190,)
(2394525764,)
(1230988107450851329,)
(813244354059993089,)
(722476963,)
(23692579,)
(1231290003860447232,)
(2358986391,)
(23677622,)
(1124864315859918849,)
(1037816383827050496,)
(17824800,)
(1174470960268926977,)
(74511714,)
(1060708562,)
(921979230824685568,)
(1549314654,)
(1199779480975396866,)
(290908083,)
(1065525162861346816,)
(121705

Check the when a twitteruser tweet a post and the numbers of likes of the post by the tweetID. 

In [370]:

c.execute("""SELECT createdTime, favoriteCount
             FROM tweet
             WHERE tweetId = 1231376918164967424
          """)
for row in c.fetchall():
    print(row)

('2020-02-23 00:35:09', 0)


Find the tweet that posted by a twitter user with id 1092534179739377666 within one day

In [371]:

c.execute("""SELECT *
             FROM tweet
             WHERE twitterUserId = 1092534179739377666 AND datetime(createdTime) > datetime('now','-1 day','localtime') 
          """)
for row in c.fetchall():
    print(row)

(1231379753132797953, 1092534179739377666, 0, '2020-02-23 00:46:25')


Find the numbers of posts that tweeted by user 1092534179739377666 within one day

In [372]:

c.execute("""SELECT COUNT(tweetId)
             FROM tweet
             WHERE twitterUserId = 1092534179739377666 AND datetime(createdTime) > datetime('now','-1 day','localtime') 
          """)
for row in c.fetchall():
    print(row)

(1,)


Find 10 most popular hastags

In [373]:

c.execute("""SELECT hashtag.text, SUM(tweet.favoriteCount)
             FROM hashtag, tweet, tweetRhashtag
             WHERE hashtag.hashtagId=tweetRhashtag.hashtagId AND tweet.tweetId=tweetRhashtag.tweetId
             GROUP BY hashtag.hashtagId
             ORDER BY SUM(tweet.favoriteCount) DESC
             LIMIT 10
          """)
for row in c.fetchall():
    print(row)

('FoxNews', 10)
('snotquiz', 4)
('eSport', 0)
('Dota2', 0)
('CSGO', 0)
('PUBG', 0)
('Hearthstone', 0)
('Fortnite', 0)
('AudibleApp', 0)
('VR', 0)


Find the popular posts(the favourite counts > the average numbers)

In [375]:

c.execute("""SELECT tweetId
             FROM tweet
             WHERE favoriteCount > (SELECT AVG(favoriteCount)
                                    FROM tweet)
          """)
for row in c.fetchall():
    print(row)

(1231372013610115077,)
(1231368083341635584,)
(1231366996882796544,)
(1231362175173873666,)
(1231360990010322944,)
(1231357468061716481,)
(1231353800717299712,)
(1231352581424926721,)
(1231351816622874625,)
(1231350909596241920,)
(1231350586928517121,)
(1231350465037815810,)
(1231348751731765249,)
(1231348558735060996,)
(1231347491871903744,)
(1231346917675716609,)
(1231345409672523776,)
(1231343138062553089,)
(1231342369162788866,)
(1231341600778936326,)
(1231339395082604550,)
(1231338076653273089,)
(1231337926317035525,)


Find the producer with most fans

In [376]:

c.execute("""SELECT personName
             FROM person
             WHERE twitterUserId IN (SELECT twitterUserId
                                     FROM twitteruser
                                     WHERE followersCount = (SELECT MAX(followersCount)
                                                             FROM twitteruser
                                                             WHERE twitterUserId IN (SELECT twitterUserId
                                                                                     FROM person
                                                                                     WHERE personId IN (SELECT DISTINCT personId
                                                                                                        FROM producer
                                                                                                        )
                                                                                    )
                                                            )
                                     AND twitterUserId IN (SELECT twitterUserId
                                                           FROM person
                                                           WHERE personId IN (SELECT DISTINCT personId
                                                                              FROM producer
                                                                             )
                                                          )
                                     )
          """)
for row in c.fetchall():
    print(row)

('Sam Houser',)


Select the game company with most fans

In [377]:

c.execute("""SELECT companyName
             FROM company
             WHERE twitterUserId IN (SELECT twitterUserId
                                     FROM twitteruser
                                     WHERE followersCount = (SELECT MAX(followersCount)
                                                             FROM twitteruser
                                                             WHERE twitterUserId IN (SELECT twitterUserId
                                                                                     FROM company
                                                                                    )
                                                             )
                                     AND twitterUserId IN (SELECT twitterUserId
                                                           FROM company
                                                          )
                                     )
          """)
for row in c.fetchall():
    print(row)

('Rockstar',)


Select the company that made Half-Life

In [378]:

c.execute("""SELECT companyName
             FROM company
             WHERE companyId IN (
                                SELECT DISTINCT companyId
                                FROM producer
                                WHERE gameId = (
                                                SELECT gameId
                                                FROM game
                                                WHERE gameName = "Half-Life"
                                               )
                                )
          """)
for row in c.fetchall():
    print(row)

('Valve',)


Select the top 10 locations of the players of Half-Life

In [384]:

c.execute("""SELECT location, COUNT(*) AS count
             FROM twitteruser
             WHERE twitteruserId IN (
                                    SELECT twitteruserId
                                    FROM person
                                    WHERE personId IN (
                                                      SELECT personId
                                                      FROM consumerRgame
                                                      WHERE gameId = (
                                                                      SELECT gameId
                                                                      FROM game
                                                                      WHERE gameName = "Half-Life"
                                                                     )
                                                     )
                                   )
             GROUP BY location
             ORDER BY count DESC
             LIMIT 10
          """)
for row in c.fetchall():
    print(row)

(None, 43)
('United States', 6)
('England, United Kingdom', 2)
('Los Angeles, CA', 2)
('Oregon, USA', 2)
('Portland, OR', 2)
('Virginia, USA', 2)
('28850', 1)
('Aldea Las anonas', 1)
('Argentina', 1)


Show all the games that Valve produced.

In [379]:

c.execute("""
             SELECT gameName
             FROM game
             WHERE gameId IN(
                             SELECT DISTINCT gameId
                             FROM producer
                             WHERE companyId = (
                                                SELECT companyId
                                                FROM company
                                                WHERE companyName = "Valve"
                                               )
                            )
          """)
for row in c.fetchall():
    print(row)

('Half-Life',)
('Half-Life 2',)


# Contribution
## BY Minghao Ru: 50%
## BY Tong Yang 50%

# Citation
https://stackoverflow.com/questions/35415469/sqlite3-unique-constraint-failed-error

https://github.com/nikbearbrown/INFO_6210/blob/master/Examples/DB_NBA.ipynb

https://github.com/nikbearbrown/INFO_6210/blob/master/Examples/A_Movie_Database.ipynb

https://developer.twitter.com/en/docs/basics/apps/overview

https://steamdb.info/

http://docs.tweepy.org/en/latest/api.html

http://docs.tweepy.org/en/latest/

https://www.sqlitetutorial.net/sqlite-python/creating-database/


